# **Import Library**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# **Load data**

In [2]:
provinsi_df = pd.read_csv('provinsi.csv')
kabupaten_df = pd.read_csv('kabupaten.csv')
makanan_khas_df = pd.read_csv('makanan_khas.csv')
image_link_df = pd.read_csv('image_link.csv')
restaurant_df = pd.read_csv('restaurant.csv')
reting_df = pd.read_csv('reting.csv')

# **Pemeriksaan Awal Data**

In [3]:
print(provinsi_df.head())
print(kabupaten_df.head())
print(makanan_khas_df.head())
print(image_link_df.head())
print(restaurant_df.head())
print(reting_df.head())

   id_provinsi   nama_provinsi
0           11            ACEH
1           12  SUMATERA UTARA
2           13  SUMATERA BARAT
3           14            RIAU
4           15           JAMBI
   id_kabupaten  id_provinsi           nama_kabupaten
0          1101           11       KABUPATEN SIMEULUE
1          1102           11   KABUPATEN ACEH SINGKIL
2          1103           11   KABUPATEN ACEH SELATAN
3          1104           11  KABUPATEN ACEH TENGGARA
4          1105           11     KABUPATEN ACEH TIMUR
   id_makanan       makanan_khas  \
0           1       Sate Bandeng   
1           2              Rabeg   
2           3         Gerem Asem   
3           4       Ketan Bintul   
4           5  Nasi Bakar Sumsum   

                                           deskripsi  
0  Sate yang terbuat dari ikan bandeng, santan, d...  
1  Makanan yang bahan dasarnya daging dan jeroan ...  
2                 Masakan dengan rasa asam yang khas  
3  Makanan dari beras ketan yang dibumbui dan dib... 

# **Penanganan Data yang Hilang**

In [4]:
# Mengisi nilai yang hilang untuk kolom numerik dengan median
for df in [provinsi_df, kabupaten_df, makanan_khas_df, image_link_df, restaurant_df, reting_df]:
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()), axis=0)

# Mengisi nilai yang hilang untuk kolom non-numerik dengan modus
for df in [provinsi_df, kabupaten_df, makanan_khas_df, image_link_df, restaurant_df, reting_df]:
    non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
    df[non_numeric_cols] = df[non_numeric_cols].apply(lambda x: x.fillna(x.mode()[0]), axis=0)

# **Penghapusan Data Duplikat**

In [5]:
provinsi_df.drop_duplicates(inplace=True)
kabupaten_df.drop_duplicates(inplace=True)
makanan_khas_df.drop_duplicates(inplace=True)
image_link_df.drop_duplicates(inplace=True)
restaurant_df.drop_duplicates(inplace=True)
reting_df.drop_duplicates(inplace=True)

# **Konversi Tipe Data**

In [6]:
restaurant_df['long'] = restaurant_df['long'].astype(float)
restaurant_df['lat'] = restaurant_df['lat'].astype(float)
reting_df['nilai'] = reting_df['nilai'].astype(float)
reting_df['jumlah_reting'] = reting_df['jumlah_reting'].astype(int)

# **Merge Data**

In [9]:
# Gabungkan restaurant dengan reting
restaurant_reting_df = pd.merge(restaurant_df, reting_df, on='id_reting', how='left')

# Gabungkan restaurant_reting dengan makanan_khas
restaurant_makanan_df = pd.merge(restaurant_reting_df, makanan_khas_df, on='id_makanan', how='left')

# Gabungkan restaurant_makanan dengan provinsi
restaurant_makanan_provinsi_df = pd.merge(restaurant_makanan_df, provinsi_df, on='id_provinsi', how='left')

# Gabungkan restaurant_makanan_provinsi dengan kabupaten
final_df = pd.merge(restaurant_makanan_provinsi_df, kabupaten_df, on=['id_kabupaten', 'id_provinsi'], how='left')

In [10]:
print(final_df.columns)

Index(['id_restaurant', 'id_reting', 'id_makanan', 'id_provinsi',
       'id_kabupaten', 'nama_tempat', 'alamat', 'long', 'lat', 'nilai',
       'jumlah_reting', 'makanan_khas', 'deskripsi', 'nama_provinsi',
       'nama_kabupaten'],
      dtype='object')


# **Normalisasi Nilai Numerik**

In [11]:
# Pilih kolom numerik yang ingin dinormalisasi
numerical_cols = ['nilai', 'jumlah_reting', 'long', 'lat']

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi kolom numerik
final_df[numerical_cols] = scaler.fit_transform(final_df[numerical_cols])

print(final_df.head())

   id_restaurant  id_reting  id_makanan  id_provinsi  id_kabupaten  \
0              1          1           1           36          3673   
1              2          2           1           36          3673   
2              3          3           1           36          3673   
3              4          4           2           36          3673   
4              5          5           2           36          3673   

                               nama_tempat  \
0           Sate Bandeng Ibu Amenah Serang   
1  Sate Bandeng - Bekakak Ayam Hj. Mariyam   
2                  Sate Bandeng Ibu Aliyah   
3   Rabeg Khas Serang H. Naswi (Magersari)   
4      Warung Dahar Rabeg Khas Wong Banten   

                                              alamat      long       lat  \
0  Jl. Sayabulu km 1, Lingk No.11, RT.2/RW.1, Dal...  0.945093  0.015711   
1  Tengah, Jl. Ki Uju No.63, RT.003/RW.002, Tenga...  0.945098  0.015903   
2  Jl. Lopang Gede III, Lopang, Kec. Serang, Kota...  0.945161  0.016022  

# **Select Fitur**

In [12]:
selected_features = ['id_restaurant', 'id_makanan', 'id_provinsi', 'id_kabupaten', 'nama_tempat', 'alamat',
                    'long', 'lat', 'nilai', 'jumlah_reting', 'makanan_khas', 'deskripsi']
final_df = final_df[selected_features]

# **Save Data**

In [13]:
final_df.to_csv('final_cleaned_normalized_data.csv', index=False)